In [1]:
import pandas as pd
import pandas_market_calendars as mcal
import datetime
import yfinance as yf
import pickle
import os

In [8]:
os.chdir('../')

In [10]:
def get_calendar():
    nyse = mcal.get_calendar('NYSE')
    a = nyse.valid_days(start_date='2014-01-01', end_date='2024-11-14')
    cal =[]
    for x in a:
        cal.append(datetime.datetime.strftime(x,'%Y-%m-%d'))
    return cal

In [33]:
data

,TFCF,VEON,BIDU,1518855D,MDLZ,SRCL,TSCO,DISH,AMZN,ALXN,...,GFS,HON,PANW,CEG,GEHC,WBD,ARM,DASH,LIN,ROP
Date,,,,,,,,,,,,,,,,,,,,,
2014-12-31,"{'Open': 35.4936, 'Close': 35.2737, 'High': 35...","{'Open': 74.797, 'Close': 70.9722, 'High': 75....","{'Open': 229.78, 'Close': 227.97, 'High': 231....","{'Open': 51.54, 'Close': 50.51, 'High': 51.68,...","{'Open': 30.1655, 'Close': 29.5912, 'High': 30...","{'Open': 133.24, 'Close': 131.08, 'High': 133....","{'Open': 67.8891, 'Close': 68.5413, 'High': 69...","{'Open': 72.8787, 'Close': 72.5998, 'High': 73...","{'Open': 15.612, 'Close': 15.518, 'High': 15.6...","{'Open': 185.81, 'Close': 185.03, 'High': 188....",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-02,"{'Open': 35.3597, 'Close': 34.9199, 'High': 35...","{'Open': 70.3772, 'Close': 72.7571, 'High': 72...","{'Open': 229.46, 'Close': 223.08, 'High': 230....","{'Open': 50.66, 'Close': 50.17, 'High': 50.78,...","{'Open': 29.6767, 'Close': 29.97, 'High': 30.0...","{'Open': 131.51, 'Close': 131.64, 'High': 132....","{'Open': 68.5239, 'Close': 67.8891, 'High': 69...","{'Open': 72.5301, 'Close': 71.9823, 'High': 73...","{'Open': 15.629, 'Close': 15.426, 'High': 15.7...","{'Open': 186.64, 'Close': 186.6, 'High': 189.3...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-05,"{'Open': 34.9008, 'Close': 34.2028, 'High': 34...","{'Open': 72.7571, 'Close': 68.6773, 'High': 74...","{'Open': 223.0, 'Close': 219.785, 'High': 224....","{'Open': 49.71, 'Close': 49.13, 'High': 49.88,...","{'Open': 29.9048, 'Close': 29.5219, 'High': 29...","{'Open': 131.02, 'Close': 130.1, 'High': 132.1...","{'Open': 67.7761, 'Close': 66.4978, 'High': 68...","{'Open': 71.6934, 'Close': 70.1596, 'High': 72...","{'Open': 15.351, 'Close': 15.11, 'High': 15.41...","{'Open': 181.3, 'Close': 182.17, 'High': 185.3...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-06,"{'Open': 34.3366, 'Close': 33.7247, 'High': 34...","{'Open': 68.3373, 'Close': 69.1023, 'High': 70...","{'Open': 220.89, 'Close': 220.18, 'High': 222....","{'Open': 49.2, 'Close': 49.21, 'High': 49.88, ...","{'Open': 29.6278, 'Close': 29.3183, 'High': 29...","{'Open': 130.36, 'Close': 130.5, 'High': 131.9...","{'Open': 66.3413, 'Close': 65.4804, 'High': 66...","{'Open': 70.0799, 'Close': 68.3966, 'High': 70...","{'Open': 15.112, 'Close': 14.765, 'High': 15.1...","{'Open': 183.66, 'Close': 177.95, 'High': 185....",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-07,"{'Open': 33.8203, 'Close': 32.4721, 'High': 33...","{'Open': 70.0372, 'Close': 68.6773, 'High': 72...","{'Open': 222.42, 'Close': 224.35, 'High': 227....","{'Open': 49.62, 'Close': 48.59, 'High': 49.64,...","{'Open': 29.4812, 'Close': 30.2388, 'High': 30...","{'Open': 130.74, 'Close': 131.3, 'High': 131.7...","{'Open': 65.6543, 'Close': 67.6022, 'High': 67...","{'Open': 68.7452, 'Close': 68.1875, 'High': 69...","{'Open': 14.875, 'Close': 14.921, 'High': 15.0...","{'Open': 181.26, 'Close': 187.93, 'High': 188....",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-08,NaN,NaN,NaN,NaN,"{'Open': 65.72, 'Close': 66.2, 'High': 66.26, ...",NaN,NaN,NaN,"{'Open': 209.75, 'Close': 208.18, 'High': 209....",NaN,...,"{'Open': 44.95, 'Close': 43.98, 'High': 44.95,...","{'Open': 218.0, 'Close': 219.49, 'High': 219.7...","{'Open': 389.0, 'Close': 391.4, 'High': 393.15...","{'Open': 242.75, 'Close': 239.37, 'High': 247....","{'Open': 88.8, 'Close': 85.94, 'High': 89.0, '...","{'Open': 9.32, 'Close': 9.18, 'High': 9.325, '...","{'Open': 148.5, 'Close': 147.48, 'High': 149.0...","{'Open': 170.4, 'Close': 171.4, 'High': 172.16...","{'Open': 465.33, 'Close': 459.48, 'High': 465....","{'Open': 575.0, 'Close': 567.62, 'High': 575.0..."
2024-11-11,NaN,NaN,NaN,NaN,"{'Open': 65.67, 'Close': 66.1, 'High': 66.565,...",NaN,NaN,NaN,"{'Open': 208.36, 'Close': 206.84, 'High': 209....",NaN,...,"{'Open': 44.43, 'Close': 42.7, 'High': 44.94, ...","{'Open': 219.91, 'Close': 225.24, 

In [11]:
cal=get_calendar()

In [12]:
data = pd.DataFrame()
for year in range(2015,2025):
    with open(f'Tools/NDX_data/NDXdata{year}.pkl', 'rb') as file:
        ndxData = pickle.load(file)
    data=pd.concat([data,ndxData])

data = data.loc[~data.index.duplicated(keep='last')]

In [13]:
def getDate(date):
    #date needs to be a string like 2024-01-01
    dateList= date.split('-')
    return datetime.date(int(dateList[0]),int(dateList[1]),int(dateList[2]) )


In [14]:
def getPrice(date, ticker, Open_Close="Close"):
    return data.loc[date,ticker][Open_Close]

In [15]:
def getDateOffset(Date,offset=0,cal=cal):
    try:
        return cal[cal.index(Date)+offset]
    except:
        cal=get_calendar()
        return cal[cal.index(Date)+offset]


In [16]:
def get_NDXmembers(year):
    file=f"Tools/NDX_data/Nasdaq100MembersJan{year}.csv"
    df = pd.read_csv(file,header = None)
    df.columns = ['members']
    NDX_members=[]
    for x in df['members']:
        ticker = x[:x.find(" ")]
        NDX_members.append(ticker)
    return NDX_members

In [17]:
def getPctChange(ticker,startDate,endDate,startTime="Close",endTime="Close"):
    startPrice = data.loc[startDate,ticker][startTime]
    endPrice = data.loc[endDate,ticker][endTime]
    pctChange = (endPrice-startPrice)/startPrice
    return pctChange

In [22]:
def get_deltaPctChange(date,ticker,debug=False):
    #a=getPctChange(ticker,getDateOffset(date,-5),getDateOffset(date,-4))
    #b=getPctChange(ticker,getDateOffset(date,-4),getDateOffset(date,-3))
    c=getPctChange(ticker,getDateOffset(date,-3),getDateOffset(date,-2))
    d=getPctChange(ticker,getDateOffset(date,-2),getDateOffset(date,-1))


   # delta_1 = (b-a)/(c-b)
    #delta_2 = (c-b)/(d-c)
    #momentum = delta_1/delta_2
    if debug:
        print(f"A:{a}\nB:{b}\nC:{c}\nD:{d}\n")
        print(f"Delta_1:{delta_1}\nDelta_2:{delta_2}\nMomentum:{momentum}")
    #return momentum
    
    return d/c

In [23]:
def get_SortedPctChange(InputDate, TimeInput=5,debug=False):
    
    NDX_members = get_NDXmembers(getDate(InputDate).year)
    PctChangeList = {"Ticker":[],"PctChange":[]}
    startDate = getDateOffset(InputDate,-TimeInput)
    endDate = getDateOffset(InputDate,-1)
    for ticker in NDX_members:
        try:     
            pctChange = getPctChange(ticker,startDate,endDate)
            PctChangeList['Ticker'].append(ticker)
            PctChangeList['PctChange'].append(pctChange)
            
            if debug:
                print("---",ticker,"----")
                print(f"Percent Change: {pctChange}")
        except:
            print("UNABLE TO FIND:---",ticker,"----")
        
    #Convert to dataframe and sort it 
    dfChange = (pd.DataFrame(PctChangeList).sort_values('PctChange',ascending=False)).reset_index(drop=True)
    
    return dfChange

In [24]:
def getShortReturns(RunDate,ShortStockList,holdTime=0,debug=False,stopLoss=-0.01):
    ShortReturnList = list()
    endDate = getDateOffset(RunDate,holdTime)
    for ticker in ShortStockList:
        try:


            #Get the data based on the provided ticker, RunDate, and startDate(being runDate-Timeinput)
            endPrice = getPrice(endDate,ticker,"Close")
            startPrice = getPrice(RunDate,ticker,"Open")
            runHighPrice= getPrice(RunDate,ticker,"High")

            Return = -(endPrice-startPrice)/startPrice
            #Checking For Stop Loss

            if -(runHighPrice-startPrice)/startPrice <= stopLoss:
                Return = stopLoss

            #Calculate and Append Return
            ShortReturnList.append(Return)

            if debug:
                print("---",ticker,"----")

                print(f"Start Date:{RunDate} , Start Price:{startPrice} ")
                print(f"End Date:{endDate} , End Price:{endPrice} ")
                print(f"Return:{Return}")
                print("")

        except:

            print("FAILED TO CALCULATE SHORT RETURN::---",ticker,"---",endDate)
           


    return ShortReturnList

In [25]:
def getLongReturns(RunDate,LongStockList,stopLoss = -0.01, holdTime=0,debug=False):
    LongReturnList = list()
    endDate = getDateOffset(RunDate,holdTime)

    for ticker in LongStockList:
        try:
            #Get the data based on the provided ticker, RunDate, and startDate(being runDate-Timeinput)
            endPrice = getPrice(endDate,ticker,"Close")
            startPrice = getPrice(RunDate,ticker,"Open")
            #Checking For Stop Loss
            runLowPrice= getPrice(endDate,ticker,"Low")
            Return = (endPrice-startPrice)/startPrice

            if (runLowPrice-startPrice)/startPrice <= stopLoss:
                Return = stopLoss


            #Calculate and Append Return

            LongReturnList.append(Return)


            if debug:
                print("---",ticker,"----")
                print(f"Start Date:{RunDate} , Start Price:{startPrice} ")
                print(f"End Date:{endDate} , End Price:{endPrice} ")
                print(f"Return:{Return}")
                print("")


        except:
            print("FAILED TO CALCULATE LONG RETURN:---",ticker,"--",endDate)
           

    return LongReturnList




In [ ]:
#PctChange = get_SortedPctChange('2024-10-09',4,debug=True)


In [32]:
data.loc['2015-12-28','1541931D']

In [26]:
#LongStockList = PctChange["Ticker"].iloc[:2]
LongReturns = getLongReturns('2015-12-18',['1541931D'],holdTime=5,debug=True)


FAILED TO CALCULATE LONG RETURN:--- 1541931D --


UnboundLocalError: cannot access local variable 'startPrice' where it is not associated with a value

In [ ]:
#LongReturns